In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [2]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Prepare training data
X = []
y = []
data_dir = "dataset"  # Folder containing 'next', 'previous', and 'none' subfolders
skipped_images = 0

In [3]:
print("Extracting landmarks from gesture images...")
for label in os.listdir(data_dir):
    label_path = os.path.join(data_dir, label)
    if os.path.isdir(label_path):
        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"Failed to load image: {img_path}, skipping...")
                skipped_images += 1
                continue

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image_rgb)

            if results.multi_hand_landmarks:
                landmarks = []
                for lm in results.multi_hand_landmarks[0].landmark:
                    landmarks.extend([lm.x, lm.y])  # Use x and y only
                X.append(landmarks)
                y.append(label)  # Label is folder name
            else:
                print(f"No hand detected in {img_path}, skipping...")
                skipped_images += 1

hands.close()

Extracting landmarks from gesture images...
No hand detected in dataset\next\WIN_20250506_15_59_56_Pro.jpg, skipping...
No hand detected in dataset\next\WIN_20250506_16_00_01_Pro (2).jpg, skipping...
No hand detected in dataset\next\WIN_20250506_16_00_14_Pro.jpg, skipping...
No hand detected in dataset\next\WIN_20250506_16_00_20_Pro (3).jpg, skipping...
No hand detected in dataset\none\WIN_20250507_09_11_10_Pro (2).jpg, skipping...
No hand detected in dataset\none\WIN_20250507_09_11_12_Pro (2).jpg, skipping...
No hand detected in dataset\none\WIN_20250507_09_11_55_Pro (2).jpg, skipping...
No hand detected in dataset\none\WIN_20250507_09_11_55_Pro (3).jpg, skipping...
No hand detected in dataset\none\WIN_20250507_09_11_55_Pro.jpg, skipping...
No hand detected in dataset\none\WIN_20250507_09_12_00_Pro.jpg, skipping...
No hand detected in dataset\none\WIN_20250507_09_12_04_Pro (2).jpg, skipping...
No hand detected in dataset\none\WIN_20250507_09_12_04_Pro.jpg, skipping...
No hand detected

In [4]:
# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Check for empty data
if len(X) == 0:
    print("No hand landmarks detected. Check your images.")
    exit()

print(f"Processed {len(X)} images successfully.")
print(f"Skipped {skipped_images} images due to loading or detection issues.")
print("Gesture classes:", np.unique(y))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")


Processed 745 images successfully.
Skipped 14 images due to loading or detection issues.
Gesture classes: ['next' 'none' 'previous']
Training set: 596 samples
Testing set: 149 samples


In [5]:
# Train the Random Forest model
try:
    print("Training Random Forest model...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    print("Model training completed.")

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy on test set: {accuracy:.2f}")
    print("Detailed results:")
    print(classification_report(y_test, y_pred))

    # Save the trained model
    joblib.dump(model, "gesture_model.pkl")
    print("Model saved as 'gesture_model.pkl'")

except Exception as e:
    print(f"An error occurred during training or saving: {e}")
    exit()

Training Random Forest model...
Model training completed.
Accuracy on test set: 0.99
Detailed results:
              precision    recall  f1-score   support

        next       1.00      1.00      1.00        36
        none       0.98      0.98      0.98        44
    previous       0.99      0.99      0.99        69

    accuracy                           0.99       149
   macro avg       0.99      0.99      0.99       149
weighted avg       0.99      0.99      0.99       149

Model saved as 'gesture_model.pkl'
